# Primitives examples

<LegacyContent>
<Admonition type="note">
This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.](https://quantum.cloud.ibm.com/docs/guides/primitives-examples)
</Admonition>
</LegacyContent>
<CloudContent>
<Admonition type="note">
This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/primitives-examples)
</Admonition>
</CloudContent>

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.0.0
qiskit-ibm-runtime~=0.37.0
```
</details>

The examples in this section illustrate some common ways to use primitives. Before running these examples, follow the instructions in [Install and set up.](install-qiskit)

<Admonition type="note">
    These examples all use the primitives from Qiskit Runtime, but you could use the base primitives instead.
</Admonition>

## Estimator examples

Efficiently calculate and interpret expectation values of the quantum operators required for many algorithms with Estimator. Explore uses in molecular modeling, machine learning, and complex optimization problems.

### Run a single experiment

Use Estimator to determine the expectation value of a single circuit-observable pair.

In [1]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
observable = SparsePauliOp("Z" * 50)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

estimator = Estimator(mode=backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: -0.22160664819944598
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


### Run multiple experiments in a single job

Use Estimator to determine the expectation values of multiple circuit-observable pairs.

In [2]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]

pubs = []
circuits = [iqp(mat) for mat in mats]
observables = [
    SparsePauliOp("X" * 50),
    SparsePauliOp("Y" * 50),
    SparsePauliOp("Z" * 50),
]

# Get ISA circuits
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

for qc, obs in zip(circuits, observables):
    isa_circuit = pm.run(qc)
    isa_obs = obs.apply_layout(isa_circuit.layout)
    pubs.append((isa_circuit, isa_obs))

estimator = Estimator(backend)
job = estimator.run(pubs)
job_result = job.result()

for idx in range(len(pubs)):
    pub_result = job_result[idx]
    print(f">>> Expectation values for PUB {idx}: {pub_result.data.evs}")
    print(f">>> Standard errors for PUB {idx}: {pub_result.data.stds}")

>>> Expectation values for PUB 0: 0.41624365482233505
>>> Standard errors for PUB 0: 0.2897573932386406
>>> Expectation values for PUB 1: 0.0234375
>>> Standard errors for PUB 1: 0.014335857671780467
>>> Expectation values for PUB 2: 0.5901639344262295
>>> Standard errors for PUB 2: 1.9749006536982647


### Run parameterized circuits

Use Estimator to run three experiments in a single job, leveraging parameter values to increase circuit reusability.

In [3]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Step 1: Map classical inputs to a quantum problem
theta = Parameter("θ")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)

number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
individual_phases = [[ph] for ph in phases]

ZZ = SparsePauliOp.from_list([("ZZ", 1)])
ZX = SparsePauliOp.from_list([("ZX", 1)])
XZ = SparsePauliOp.from_list([("XZ", 1)])
XX = SparsePauliOp.from_list([("XX", 1)])
ops = [ZZ, ZX, XZ, XX]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
chsh_isa_circuit = pm.run(chsh_circuit)
isa_observables = [
    operator.apply_layout(chsh_isa_circuit.layout) for operator in ops
]

# Step 3: Execute using Qiskit primitives.

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((4, 1))

estimator = Estimator(backend, options={"default_shots": int(1e4)})
job = estimator.run([(chsh_isa_circuit, reshaped_ops, individual_phases)])
# Get results for the first (and only) PUB
pub_result = job.result()[0]
print(f">>> Expectation values: {pub_result.data.evs}")
print(f">>> Standard errors: {pub_result.data.stds}")
print(f">>> Metadata: {pub_result.metadata}")

>>> Expectation values: [[ 1.00968373  0.95443304  0.80928676  0.58375525  0.29527418 -0.02332304
  -0.32742827 -0.60730472 -0.82083506 -0.95896179 -1.00311705 -0.95103648
  -0.80181433 -0.58783112 -0.28417876  0.02196441  0.31135123  0.60798404
   0.82174081  0.96462272  1.01172167]
 [ 0.00656668  0.33218346  0.59688861  0.82740174  0.97005721  1.00674005
   0.97051009  0.80747526  0.57628282  0.31293629  0.00158506 -0.3190501
  -0.60617254 -0.8307983  -0.95918822 -1.0040228  -0.94650774 -0.79773846
  -0.59122768 -0.29844431  0.02037935]
 [-0.03192765 -0.34327888 -0.62360821 -0.84166729 -0.97503883 -1.02032628
  -0.97345377 -0.82015575 -0.59937942 -0.31135123  0.0194736   0.33376852
   0.61590934  0.8369121   0.98749288  1.03617689  0.96054685  0.82264656
   0.58556675  0.29776499 -0.02626672]
 [ 1.03300677  0.96892502  0.81925     0.59009549  0.30886042 -0.01766211
  -0.3634318  -0.61160703 -0.8405351  -0.98115264 -1.01308029 -0.96484915
  -0.81019251 -0.57945294 -0.30297305  0.03192

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

<CloudContent>
<Admonition type="caution">
The following code block will return an error for Open Plan users because it uses sessions. Open Plan workloads can run only in [job mode](/docs/guides/execution-modes#job-mode) or [batch mode](/docs/guides/execution-modes#batch-mode).
</Admonition>
</CloudContent>

In [4]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    EstimatorV2 as Estimator,
)

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = iqp(mat)
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = iqp(mat)
observable = SparsePauliOp("X" * 50)
another_observable = SparsePauliOp("Y" * 50)

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
another_isa_observable = another_observable.apply_layout(
    another_isa_circuit.layout
)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    estimator.options.resilience_level = 1

    job = estimator.run([(isa_circuit, isa_observable)])
    another_job = estimator.run(
        [(another_isa_circuit, another_isa_observable)]
    )
    result = job.result()
    another_result = another_job.result()

    # first job
    print(f" > Expectation value: {result[0].data.evs}")
    print(f" > Metadata: {result[0].metadata}")

    # second job
    print(f" > Another Expectation value: {another_result[0].data.evs}")
    print(f" > More Metadata: {another_result[0].metadata}")

 > Expectation value: 0.02001953125
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}
 > Another Expectation value: -0.09828872312417727
 > More Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


<span id="sampler-examples"></span>
## Sampler examples

Generate entire error-mitigated quasi-probability distributions sampled from quantum circuit outputs. Leverage Sampler’s capabilities for search and classification algorithms like Grover’s and QVSM.

### Run a single experiment

Use Sampler to return the measurement outcome as bitstrings or counts of a single circuit.

In [5]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]

print(f" > First ten results: {pub_result.data.meas.get_bitstrings()[:10]}")

 > First ten results: ['1001010100000000011000010001011001111000010010100010110101001010111111111010010100101100100001001010000100100000011001100001001', '0000110001010100010011101001000010101011001100111010110100100011101111011010000100000000000001101000000111110010111110100000001', '1011111001010011000011111111001000001011011110100111111100101001010111111010010000010011101100111010000111000010001000100000011', '0110001111011001000001001110100010011111010101000110010110000101010001111001000010111000111000001100000001010000001001101000111', '1110110100110011011011101000001001100001001000001001110111100001000001010111100000011100011001101110001010010010001010001010001', '0111111111110001000101000010011100000000001111000011001101101001011001110000000101110000000010100000101100000000011111000000011', '1001010001011001001000101100010111001110000010100110000010100001011100110000010110010000110110110101001010000000011101000000001', '11001101000000000101101010000000110001010000110001111000111

### Run multiple experiments in a single job

Use Sampler to return the measurement outcome as bitstrings or counts of multiple circuits in one job.

In [6]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]
circuits = [iqp(mat) for mat in mats]
for circuit in circuits:
    circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuits = pm.run(circuits)

sampler = Sampler(mode=backend)
job = sampler.run(isa_circuits)
result = job.result()

for idx, pub_result in enumerate(result):
    print(
        f" > First ten results for pub {idx}: {pub_result.data.meas.get_bitstrings()[:10]}"
    )

 > First ten results for pub 0: ['0101000100000101100101100100100010111010011000110001011110011010111000001101000010000110000000010001011001101000100101010010011', '0001110100110010101101010110101101101100000100110110010110110010001110010100000110101100010000010101100000100010100000011110011', '1100010100111010000000110110011001110100100101001011001101010111000101010000110010000110110000000001001010001101101101001110010', '1000010100011011101000100110001111110011010101100000101001011000000101011100100111000100011001001101000000101010101111000010110', '1011011010000111101001000000000101000100010000110000100101010000000101000100010110100110100000010100110001001011001001111000110', '0010001100010010110010010010101100011100100000000010001100011010010100011000001100001110010000110000001000100111001001010100100', '0110000111010010101101110111000111001100000100001001111011011010000100000011011011100111011000011101001000010001000011011100101', '1111000010101011101001110010010110111100100000110

### Run parameterized circuits

Run several experiments in a single job, leveraging parameter values to increase circuit reusability.

In [7]:
import numpy as np
from qiskit.circuit.library import real_amplitudes
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

# Step 1: Map classical inputs to a quantum problem
circuit = real_amplitudes(num_qubits=n_qubits, reps=2)
circuit.measure_all()

# Define three sets of parameters for the circuit
rng = np.random.default_rng(1234)
parameter_values = [
    rng.uniform(-np.pi, np.pi, size=circuit.num_parameters) for _ in range(3)
]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

# Step 3: Execute using Qiskit primitives.
sampler = Sampler(backend)
job = sampler.run([(isa_circuit, parameter_values)])
result = job.result()
# Get results for the first (and only) PUB
pub_result = result[0]
# Get counts from the classical register "meas".
print(
    f" >> First ten results for the meas output register: {pub_result.data.meas.get_bitstrings()[:10]}"
)

 >> First ten results for the meas output register: ['0001100111011010011001101001001111100000000000001010111011101010111010101000000110110000101011111011000100011100110010000101100', '0000111101110101110001001100110110110001100111011111011010011100010011010001001101000010000000111100101100011001100011111110110', '1100011001011001000101010101101011000111011010011110101000101100010011100101011010110100001110100011111100101101000011000100010', '0001001000000011101111010111001011100001101111111010111000111101110111101111101000010011011001101011101001000000001001100000111', '0101110101010101000111110101110001100010001010010111101111111101101000111101101101010000101000001110010001100010100001011010110', '0001110010010011110011100100000110000010111010011001111000011001011101011111011010111010101111011011010000110111111101011110111', '0000110011011101100000000101101011010001011010000001111001101011010100100111011110111001011100011111011011010011001001000101101', '10010010010110111110011111001

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

<CloudContent>
<Admonition type="caution">
The following code block will return an error for users on the Open Plan, because it uses sessions. Workloads on the Open Plan can run only in [job mode](/docs/guides/execution-modes#job-mode) or [batch mode](/docs/guides/execution-modes#batch-mode).
</Admonition>
</CloudContent>

In [8]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.quantum_info import random_hermitian
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = iqp(mat)
circuit.measure_all()
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = iqp(mat)
another_circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run([isa_circuit])
    another_job = sampler.run([another_isa_circuit])
    result = job.result()
    another_result = another_job.result()

# first job

print(
    f" > The first ten measurement results of job 1: {result[0].data.meas.get_bitstrings()[:10]}"
)

 > The first ten measurement results of job 1: ['0111100010110101010101111101110111010110010010110100000011011011100000101110011001011010111000000001111010100000010000100001101', '0000000011110010000000000111010100000110001111000010010101010100101110011111010101111000100010101010010011100001010010000011110', '0011000000000100001110011110000100110011010001001100000111000011111110011001100001000100000010010000010001000001101000001110000', '1000111100000110011010100111111101001101011111101110010000011000100000110001100000001000101100110100011001101000000000011111001', '1110000001111100001101001000011001100010001110000010000110001010000000100111100111111110001010001101110101101001111010001101000', '0010110010000010000100110001010001110001000110111010010000011001101100101000101011110010100000001010000010001101001000110001000', '0011000001110100010101100010010011111010000010001100101110001010000101010000011000011000111001011000000100000000001011101101001', '1110111100000100011010111101001110

In [9]:
# second job
print(
    " > The first ten measurement results of job 2:",
    another_result[0].data.meas.get_bitstrings()[:10],
)

 > The first ten measurement results of job 2: ['0000001010111111011011010000100101001110101100110010101110010110100100000010111111000001000110010110111001011010010001100000000', '0011111111011110011110101000001111110101000010010010100010101010000100000011000010011010011111010000001111111111010001100010000', '1010100100111001111100011101111011010101101010010010111010001110110100101010000110111001000000000001000101001000101011111001010', '0011111011100001110110110010010100101110100101000110100111100110100010000011100000001000000010101000101000001100010010000000100', '0100000001000111000000011001000111010110100001000101111000100000110111001000110100101111011110000100110101011010010000110100111', '0110111000000101011110010010010111101010100101110010000010100000111101100011010010101100100011101000000110010000100101110010010', '1001111000011100001110010011101000000001110000111010101001111100100100000001000000110000100010110010010000001011000000100010010', '1011101110111110100101110000001100

## Next steps

<Admonition type="tip" title="Recommendations">
    - [Specify advanced runtime options.](runtime-options-overview)
    - Practice with primitives by working through the [Cost function lesson](https://learning.quantum.ibm.com/course/variational-algorithm-design/cost-functions#primitives) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile/) section.
    - Try the [Submit pre-transpiled circuits](https://learning.quantum.ibm.com/tutorial/submitting-user-transpiled-circuits-using-primitives) tutorial.
    - Read [Migrate to V2 primitives](/docs/migration-guides/v2-primitives).
    - Understand the [Job limits](/docs/guides/job-limits) when sending a job to an IBM&reg; QPU.
</Admonition>